In [111]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt 
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lubab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lubab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [113]:
data = pd.read_csv("C:\\Users\\lubab\\OneDrive\\Documents\\Data Science Portfolio Projects\\Healthcare Data Anayst Jobs Australia\\healthcare-data-analyst-jobs-australia\\data\\raw\\healthcare_jobs_full.csv")
len(data)

110

In [115]:
#removing expired jobs 
data = data[data['full description'] != 'Job description not found or job no longer advertised']
len(data)

103

In [117]:
data = data.drop_duplicates()
len(data)

103

In [119]:
data.shape
print(data.columns)

Index(['Title', 'Company', 'Location', 'Short Description', 'Link',
       'full description'],
      dtype='object')


In [121]:
data['full description'].head(3)

0    Location:\nSunshine Hospital\nEmployment Type:...
1    We're Hiring: Population Health Data Analyst\n...
2    Location:\nWilliamstown Hospital\nEmployment T...
Name: full description, dtype: object

In [123]:
#turning everything into lowercase
data['full description'] = data['full description'].str.lower()
data['full description'].head(3)

0    location:\nsunshine hospital\nemployment type:...
1    we're hiring: population health data analyst\n...
2    location:\nwilliamstown hospital\nemployment t...
Name: full description, dtype: object

In [125]:
data.head()

,Title,Company,Location,Short Description,Link,full description
0,Data Analyst,Western Health,St Albans,A unique opportunity exists to join the Wester...,https://www.seek.com.au/job/86244018?type=stan...,location:\nsunshine hospital\nemployment type:...
1,Population Health Data Analyst,South Eastern Melbourne Primary Health Network,Heatherton,Analyse data to drive population health planning,https://www.seek.com.au/job/86312389?type=stan...,we're hiring: population health data analyst\n...
2,Data Analyst,Western Health,Williamstown,Drive data-driven insights and improve organis...,https://www.seek.com.au/job/86238954?type=stan...,location:\nwilliamstown hospital\nemployment t...
3,Data Analyst,Western Health,Footscray,"Excellent data management and problem skills, ...",https://www.seek.com.au/job/86244700?type=stan...,about you:\nexcellent data management and prob...
4,Senior Analyst,Alfred Health,Melbourne VIC,Permanent Full Time (1.0 EFT / 80 Hours) + ADO...,https://www.seek.com.au/job/86301836?type=stan...,alfred health\nalfred health is a leader in he...


In [127]:
#removing whitespace
data['full description'] = data['full description'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())
data['full description'].head(3)

0    location: sunshine hospital employment type: f...
1    we're hiring: population health data analyst l...
2    location: williamstown hospital employment typ...
Name: full description, dtype: object

In [129]:
#removing punctuation
data['full description'] = data['full description'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)).strip())
data['full description'].head(3)

0    location sunshine hospital employment type fix...
1    were hiring population health data analyst loc...
2    location williamstown hospital employment type...
Name: full description, dtype: object

In [131]:
#tokenizing sentences
data['tokenized_sent'] = data['full description'].apply(lambda x: word_tokenize(str(x))) 

#removing stop words
stop_words = set(stopwords.words('english'))
data['token_no_stopwords'] = data['tokenized_sent'].apply(lambda x: [word for word in x if word not in stop_words])